<a href="https://colab.research.google.com/github/nkcong206/RAG_tutorial/blob/main/GraphRAG%2BCamelAI%2BNeo4j.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![](https://storage.googleapis.com/mle-courses-prod/users/61b6fa1ba83a7e37c8309756/private-files/d4d15b20-4fc0-11ef-bf69-71eafa46c86b-RAGGraph___Neo4J___CamelAI.png)

Cài đặt thư viện

In [ ]:
!pip install -q camel-ai[all]

In [ ]:
from camel.models import ModelFactory
from camel.types import ModelPlatformType, ModelType
from camel.configs import MistralConfig, OllamaConfig, ChatGPTConfig
from camel.loaders import UnstructuredIO
from camel.storages import Neo4jGraph
from camel.retrievers import AutoRetriever
from camel.embeddings import MistralEmbedding, OpenAIEmbedding
from camel.types import StorageType, RoleType
from camel.agents import ChatAgent, KnowledgeGraphAgent
from camel.messages import BaseMessage

In [ ]:
import os
from getpass import getpass
from google.colab import userdata
# Prompt for the API key securely
os.environ["OPENAI_API_KEY"] = userdata.get("open_ai_key")

Kết nối tới Neo4J

Bạn hãy lấy URL, username và password của Neo4J [qua đây ](https://console.neo4j.io/)

In [ ]:
# Set Neo4j instance
n4j = Neo4jGraph(
    url="<URL>",
    username="<USERNAME>",
    password="<PASSWORD>",
)

In [ ]:
# Set up model
gpt4 = ModelFactory.create(
    model_platform=ModelPlatformType.OPENAI,
    model_type=ModelType.GPT_4,
    model_config_dict=ChatGPTConfig(temperature=0.2).__dict__,
)

In [ ]:
# Set instance
uio = UnstructuredIO()
kg_agent = KnowledgeGraphAgent(model=gpt4)

In [ ]:
text_example = """
Hưng Đạo Vương Trần Quốc Tuấn là người đã trực tiếp chỉ huy quân đội đánh tan hai cuộc xâm lược của quân Nguyên–Mông năm 1285 và năm 1288. Ông được biết đến với những chiến công lẫy lừng, là một nhà quân sự tài ba với gia thế hiển hách. Đúng là người giỏi, người tài nên cái gì ông cũng dám làm, dám chịu. Đau khổ vì người con gái mình yêu thương phải lấy người khác làm chồng, ông đã tạo ra một phi vụ cướp dâu chấn động nhất lịch sử Việt Nam.


Năm 1237, Trần Cảnh (Trần Thái Tông) lên ngôi đã lâu mà vẫn không có con nối dõi, lo sợ nhà Trần bị tuyệt hậu, Trần Thủ Độ đã gây sức ép phế Lý Chiêu Hoàng để ép Trần Cảnh kết hôn với chị dâu của mình là công chúa Thuận Thiên, tức là vợ của Trần Liễu - cha của Trần Quốc Tuấn. Trần Liễu tức giận, mang binh rửa hận nhưng thân già sức yếu nên việc bất thành, cuối cùng phải buông giáp quy hàng, bị giáng xuống làm An Sinh Vương, cho về an trú ở đất Yên Sinh. Khi ấy, Trần Quốc Tuấn mới 7 tuổi.

Chị của Trần Cảnh là Thụy Bà công chúa vì thương cháu mình đang còn nhỏ phải rời kinh đô tới nơi xa, đã cầu xin vua để nhận nuôi Quốc Tuấn để khuây khỏa nỗi buồn khi chồng bà đã mất.

Bà nhận nuôi Quốc Tuấn được 8 năm, cho ông học văn, học võ, lớn lên với các con em hoàng tộc cùng trang lứa. Cũng chính trong thời gian này, Trần Quốc Tuấn gặp gỡ, cùng trải qua thời niên thiếu của mình với Thiên Thành công chúa.

Trong suốt những năm tháng học tập và sinh sống nơi cung cấm, tình cảm của công chúa và Trần Quốc Tuấn cứ lớn dần lên, quấn quýt không rời. Mối tình thanh mai trúc mã cứ thế nở rộ.

Cứ tưởng đây là mối lương duyên trời ban, cho tới khi Thiên Thành tới tuổi gả chồng, vua Trần Thái Tông đã hạ chỉ gả nàng cho Trung Thành Vương, con trai của Nhân Đạo Vương, phá tan giấc mộng đôi lứa của hai người.

Sách Đại Việt sử ký toàn thư ghi lại rằng ngày 15 tháng 2 năm 1251, vua mở hội lớn 7 ngày đêm, bày các tranh về lễ kết tóc và nhiều trò chơi cho người trong triều ngoài nội đến xem, ý muốn cho công chúa Thiên Thành làm lễ kết tóc với Trung Thành Vương. Trước đó, nhà vua cũng cho Thiên Thành công chúa về ở vương phủ cha của Trung Thành Vương để chờ ngày làm lễ ăn hỏi.

Trong khi cả kinh thành đang tưng bừng với những trò chơi và lễ hội, ở vương phủ Trần Quốc Tuấn chỉ cần nghĩ đến việc ngày mai, người con gái mình yêu thương sẽ trở thành vợ người khác thì tâm tư của chàng càng đau đớn. Chàng trằn trọc suốt đêm không ngủ cuối cùng chàng đưa ra quyết định táo bạo đó chính là đột nhập vào phủ Nhân Đạo Vương, cướp vợ về.

Nghĩ là làm, trong đêm tối, nhân lúc mọi người còn đang say mê với lễ hội, Trần Quốc Tuấn lẻn vào phủ Nhân Đạo Vương. Biết không thể theo vào bằng cửa chính, chàng đã tìm cách trèo tường, vượt qua hàng toán lính tuần tra, dò trong đêm đen và tìm được chính xác phòng công chúa.

Trái tim đau khổ của Thiên Thành sống lại lần nữa khi thấy người tình trong mộng xuất hiện trước mặt mình. Khi ấy, cả phủ Nhân Đạo Vương vẫn đang say trong lễ hội, không ai biết, trong phòng công chúa, đôi uyên ương đã gặp lại nhau. Thế nhưng, sự liều lĩnh này của Trần Quốc Tuấn sẽ trở thành thảm án nếu sự vụ bị bại lộ. Và nếu như chuyện không bại lộ, thì hôm sau công chúa Thiên Thành phải kết hôn với con trai của Nhân Đạo Vương. Để tránh khỏi tai ương đó, Trần Quốc Tuấn đã đi tiếp một bước cờ cao minh, đó chính là dồn nhà vua vào thế sự đã rồi.

Ngay sau khi đột nhập thành công vào phòng công chúa, việc đầu tiên Trần Quốc Tuấn làm là ra lệnh cho thị nữ của công chúa về báo cho Thụy Bà công chúa, mẹ nuôi của chàng, Sau khi nhận được tin báo, Thụy Bà công chúa vào cung ngay lập tức và than khóc với Thái Tông: "Không ngờ Quốc Tuấn càn rỡ đang đêm lẻn vào chỗ của Thiên Thành. Nhân Đạo Vương đã bắt giữ hắn rồi, e sẽ giết hắn mất. Xin bệ hạ rủ lòng thương, sai người đến cứu".

Lời nói của Thụy Bà công chúa như sét đánh ngang tai nhà vua, Trần Thái Tông tức Cảnh lúc bấy giờ đã nhận đủ lễ vật của Nhân Đạo Vương, sao có thể để Trần Quốc Tuấn cả gan làm loạn như vậy? Thụy Bà công chúa tiếp tục kiên trì van xin. Cộng thêm với việc ông nghĩ rằng đó là huyết mạch của anh trai Trần Liễu, Thái Tông đã đã sai người vây phủ Nhân Đạo Vương, xông thẳng tới hoa viên vắng lặng, vào phòng công chúa Thiên Thành để áp giải, thực chất là hộ tống, Trần Quốc Tuấn ra ngoài một cách an toàn. Đến lúc đó, cả phủ Nhân Đạo Vương mới ngỡ ngàng nhận ra Trần Quốc Tuấn đã vào phủ "tư thông" với công chúa Thiên Thành.

Việc công chúa "tư thông" với nam tử khác ngay trong phủ sắp cưới là điều không thể chấp nhận được. Hôm sau, Thụy Bà công chúa đã nhanh tay hỏi cưới công chúa Thiên Thành cho cháu trai mình, với sinh lễ là 10 mâm vàng sống và nói "vì vội quá nên không sắm đủ lễ vật, mong hoàng thượng nhận cho". Trước chuyện đã rồi, Trần Thái Tông đành xuống chiếu gả Thiên Thành công chúa cho Trần Quốc Tuấn và ngậm ngùi cắt 2.000 khoảnh ruộng tốt ở huyện Ứng Thiên để "an ủi" Nhân Đạo Vương.

Cuối cùng Trần Quốc Tuấn chàng đã lấy được thanh mai trúc mã mà ông yêu bấy lâu. Hai vợ chồng chàng đã có một cuộc sống êm ấm, hạnh phúc, sinh được bốn trai, một gái. Bốn người con trai ai cũng không phụ danh tiếng người cha, đều là những danh tướng lẫy lừng nhà Trần. Người con gái út sau này trở thành Bảo Thánh Hoàng Hậu Trần Trinh, vợ vua Trần Nhân Tông, mẹ đẻ vua Trần Anh Tông.


"""

Tạo đồ thị

In [ ]:
# Create an element from given text
element_example = uio.create_element_from_text(text=text_example)

In [ ]:
# Check graph element
graph_elements = kg_agent.run(element_example, parse_graph_elements=True)
print(graph_elements)

GraphElement(nodes=[Node(id='Hưng Đạo Vương Trần Quốc Tuấn', type='Person', properties={'source': 'agent_created'}), Node(id='quân Nguyên–Mông', type='Organization', properties={'source': 'agent_created'}), Node(id='Trần Cảnh', type='Person', properties={'source': 'agent_created'}), Node(id='Trần Thái Tông', type='Person', properties={'source': 'agent_created'}), Node(id='Lý Chiêu Hoàng', type='Person', properties={'source': 'agent_created'}), Node(id='công chúa Thuận Thiên', type='Person', properties={'source': 'agent_created'}), Node(id='Trần Liễu', type='Person', properties={'source': 'agent_created'}), Node(id='An Sinh Vương', type='Title', properties={'source': 'agent_created'}), Node(id='đất Yên Sinh', type='Location', properties={'source': 'agent_created'}), Node(id='Thụy Bà công chúa', type='Person', properties={'source': 'agent_created'}), Node(id='Thiên Thành công chúa', type='Person', properties={'source': 'agent_created'}), Node(id='Trung Thành Vương', type='Person', proper

Thêm đồ thị vào Database

In [ ]:
# Add the element to neo4j database
n4j.add_graph_elements(graph_elements=[graph_elements])

In [ ]:
query="Mối quan hệ giữa Trần Cảnh và Lý Chiêu Hoàng là gì?"

In [ ]:
# Create an element from user query
query_element = uio.create_element_from_text(text=query)

# Let Knowledge Graph Agent extract node and relationship information from the qyery
ans_element = kg_agent.run(query_element, parse_graph_elements=True)

In [ ]:
ans_element

GraphElement(nodes=[Node(id='Trần Cảnh', type='Person', properties={'source': 'agent_created'}), Node(id='Lý Chiêu Hoàng', type='Person', properties={'source': 'agent_created'})], relationships=[Relationship(subj=Node(id='Trần Cảnh', type='Person', properties={'source': 'agent_created'}), obj=Node(id='Lý Chiêu Hoàng', type='Person', properties={'source': 'agent_created'}), type='MarriedTo', properties={'source': 'agent_created'})], source=<unstructured.documents.elements.Text object at 0x78cfe53e5000>)

Chuyển prompt thành truy vấn trên Databse

In [ ]:

# Match the enetity got from query in the knowledge graph storage content
kg_result = []
for node in ans_element.nodes:
    n4j_query = f"""
MATCH (n {{id: '{node.id}'}})-[r]->(m)
RETURN 'Node ' + n.id + ' (label: ' + labels(n)[0] + ') has relationship ' + type(r) + ' with Node ' + m.id + ' (label: ' + labels(m)[0] + ')' AS Description
UNION
MATCH (n)<-[r]-(m {{id: '{node.id}'}})
RETURN 'Node ' + m.id + ' (label: ' + labels(m)[0] + ') has relationship ' + type(r) + ' with Node ' + n.id + ' (label: ' + labels(n)[0] + ')' AS Description
"""
    print(n4j_query)
    result = n4j.query(query=n4j_query)

    kg_result.extend(result)


MATCH (n {id: 'Trần Cảnh'})-[r]->(m)
RETURN 'Node ' + n.id + ' (label: ' + labels(n)[0] + ') has relationship ' + type(r) + ' with Node ' + m.id + ' (label: ' + labels(m)[0] + ')' AS Description
UNION
MATCH (n)<-[r]-(m {id: 'Trần Cảnh'})
RETURN 'Node ' + m.id + ' (label: ' + labels(m)[0] + ') has relationship ' + type(r) + ' with Node ' + n.id + ' (label: ' + labels(n)[0] + ')' AS Description


MATCH (n {id: 'Lý Chiêu Hoàng'})-[r]->(m)
RETURN 'Node ' + n.id + ' (label: ' + labels(n)[0] + ') has relationship ' + type(r) + ' with Node ' + m.id + ' (label: ' + labels(m)[0] + ')' AS Description
UNION
MATCH (n)<-[r]-(m {id: 'Lý Chiêu Hoàng'})
RETURN 'Node ' + m.id + ' (label: ' + labels(m)[0] + ') has relationship ' + type(r) + ' with Node ' + n.id + ' (label: ' + labels(n)[0] + ')' AS Description



In [ ]:
kg_result = [item['Description'] for item in kg_result]

# Show the result from knowledge graph database
print(kg_result)


['Node Trần Cảnh (label: Person) has relationship MARRIED with Node Lý Chiêu Hoàng (label: Person)', 'Node Trần Cảnh (label: Person) has relationship MARRIED with Node công chúa Thuận Thiên (label: Person)']


In [ ]:
comined_results = "\n".join(kg_result)

Kết hợp prompt và tri thức từ Database

In [ ]:
comined_results

'Node Trần Cảnh (label: Person) has relationship MARRIED with Node Lý Chiêu Hoàng (label: Person)\nNode Trần Cảnh (label: Person) has relationship MARRIED with Node công chúa Thuận Thiên (label: Person)'

Đưa prompt này qua LLMs và lấy kết quả

In [ ]:

# Set agent
sys_msg = BaseMessage.make_assistant_message(
    role_name="CAMEL Agent",
    content="""You are a helpful assistant to answer question,
        I will give you the Original Query and Retrieved Context,
    answer the Original Query based on the Retrieved Context.""",
)

camel_agent = ChatAgent(system_message=sys_msg,
                        model=gpt4)

# Pass the retrieved infomation to agent
user_prompt=f"""
The Original Query is {query}
The Retrieved Context is {comined_results}
"""

user_msg = BaseMessage.make_user_message(
    role_name="CAMEL User", content=user_prompt
)

# Get response
agent_response = camel_agent.step(user_msg)

print(agent_response.msg.content)

Trần Cảnh có mối quan hệ là đã kết hôn với Lý Chiêu Hoàng.


In [ ]:
user_prompt

'\nThe Original Query is Mối quan hệ giữa Trần Cảnh và Lý Chiêu Hoàng là gì?\nThe Retrieved Context is Node Trần Cảnh (label: Person) has relationship MARRIED with Node Lý Chiêu Hoàng (label: Person)\nNode Trần Cảnh (label: Person) has relationship MARRIED with Node công chúa Thuận Thiên (label: Person)\n'